분석에 필요한 패키지를 불러옵니다.

In [4]:
import numpy as np
import pandas as pd

원본 데이터를 불러옵니다.

In [5]:
fog_train = pd.read_csv('../data/fog_train.csv')
fog_test = pd.read_csv('../data/fog_test.csv')

편의를 위해 train과 test 열 이름을 통일합니다.<br>
예측 및 제출 시 원본데이터의 열이름을 다시 적용합니다.

In [6]:
# 열 이름 앞의 접미사를 제거하는 함수
def remove_prefix(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    # prefix: "fog_train." , "fog_test."
    res_list = [] # 접미사가 제거된 열이름을 담는 리스트
    n = len(prefix) # 제거될 접미사의 길이

    for col in df.columns:
        if prefix == col[:n]: # 열 이름에 접미사가 있는지 파악
            res_list.append(col[n:]) # 접미사가 제거된 열이름 저장
        else:
            res_list.append(col) # 접미사가 없으므로 그대로 열이름 저장

    df.columns = res_list # 열 이름을 변경

    return df

train_data = remove_prefix(fog_train, "fog_train.")
test_data = remove_prefix(fog_test, "fog_test.")

In [7]:
# 결과 확인
print(train_data.columns)
print(test_data.columns)

Index(['year', 'month', 'day', 'time', 'minute', 'stn_id', 'ws10_deg',
       'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts', 'vis1', 'class'],
      dtype='object')
Index(['year', 'month', 'day', 'time', 'minute', 'stn_id', 'ws10_deg',
       'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts', 'class'],
      dtype='object')


-99.9, -99가 결측치 표시라고 판단하여 결측치(NaN)로 대체했습니다.

In [8]:
def convert_nan(x):
    if x == -99 or x == -99.9:
        return np.nan
    return x
for col in train_data.columns:
    train_data[col] = train_data[col].apply(convert_nan)

지역 구분을 위해 stn_id를 첫 번째 글자(FirstLetter)와 두 번째 글자(SecondLetter)로 구분하여 열로 지정했습니다.

In [9]:
def gen_first_letter(stn):
    return stn[0]
def gen_second_letter(stn):
    return stn[1]

train_data['FirstLetter'] = train_data['stn_id'].apply(gen_first_letter)
train_data['SecondLetter'] = train_data['stn_id'].apply(gen_second_letter)
test_data['FirstLetter'] = test_data['stn_id'].apply(gen_first_letter)
test_data['SecondLetter'] = test_data['stn_id'].apply(gen_second_letter)

In [10]:
# 편의를 위해 stn_id 다음으로 FirstLetter, SecondLetter가 올 수 있게 열 순서를 변경
train_data = train_data[['year', 'month', 'day', 'time', 'minute', 'stn_id', 'FirstLetter', 'SecondLetter',
                         'ws10_deg', 'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts', 'vis1', 'class']]
test_data = test_data[['year', 'month', 'day', 'time', 'minute', 'stn_id', 'FirstLetter', 'SecondLetter',
                       'ws10_deg', 'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts', 'class', ]]

sun10 변수의 이상치를 처리합니다<br>
1. sun10의 값이 1 이상인 경우 결측치로 대체<br>
2. 야간(20:00 ~ 05:00)에 sun10의 값이 0이 아닌 경우 0으로 대체

In [11]:
# 1. sun10의 값이 1 이상인 경우 결측치로 대체
train_data.loc[train_data['sun10'] > 1]

,year,month,day,time,minute,stn_id,FirstLetter,SecondLetter,ws10_deg,ws10_ms,ta,re,hm,sun10,ts,vis1,class
904125,I,2,26,18,30,EA,E,A,79.9,2.9,9.8,0.0,76.7,9.63,10.3,19537.0,4.0
1255059,J,10,28,19,50,AD,A,D,19.1,1.2,12.2,0.0,66.9,26.87,9.7,24294.0,4.0
1494527,J,5,19,19,10,BA,B,A,135.1,0.8,21.7,0.0,66.4,14.98,22.2,31400.0,4.0
1521613,J,11,23,21,30,BA,B,A,310.0,4.2,1.3,0.0,61.2,5.76,1.2,50000.0,4.0
1718046,J,8,20,0,20,CA,C,A,221.1,0.8,21.6,0.0,93.3,14.00,22.0,50000.0,4.0
1718047,J,8,20,0,30,CA,C,A,228.8,1.2,21.5,0.0,93.2,12.00,21.9,50000.0,4.0
2276999,K,4,8,15,20,AD,A,D,228.6,2.0,22.9,0.0,19.2,20.83,35.1,20000.0,4.0
2532439,K,2,16,12,40,BA,B,A,294.2,6.1,-3.5,0.0,29.5,8.96,-0.8,48400.0,4.0
2592053,K,4,6,12,20,BB,B,B,246.3,4.7,17.0,0.0,28.3,11.68,30.2,37868.0,4.0
3097358,K,11,16,13,50,EB,E,B,303.7,1.0,16.7,0.0,34.8,1.57,33.5,38393.0,4.0


In [12]:
train_data.loc[train_data['sun10'] > 1, 'sun10'] = np.nan
test_data.loc[test_data['sun10'] > 1, 'sun10'] = np.nan

In [13]:
# 2. 야간(20:00 ~ 05:00)에 sun10의 값이 0이 아닌 경우 0으로 대체
train_data['sun10'].loc[((train_data['time'] >= 20) | (train_data['time'] <= 4) | ((train_data['time'] == 5) & (train_data['minute'] == 0))) &
                        (train_data['sun10'] > 0)]

15551      0.01
17399      0.01
17415      0.01
17453      0.01
17559      0.01
           ... 
3129819    0.01
3134417    0.01
3139608    0.01
3149392    0.01
3151707    0.01
Name: sun10, Length: 52436, dtype: float64

In [14]:
train_data['sun10'].loc[((train_data['time'] >= 20) | (train_data['time'] <= 4) | ((train_data['time'] == 5) & (train_data['minute'] == 0))) &
                        (train_data['sun10'] > 0)] = 0.0
test_data['sun10'].loc[((test_data['time'] >= 20) | (test_data['time'] <= 4) | ((test_data['time'] == 5) & (test_data['minute'] == 0))) &
                       (test_data['sun10'] > 0)] = 0.0

C:\Users\JungHwan\AppData\Local\Temp\ipykernel_23680\721074280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['sun10'].loc[((train_data['time'] >= 20) | (train_data['time'] <= 4) | ((train_data['time'] == 5) & (train_data['minute'] == 0))) &
C:\Users\JungHwan\AppData\Local\Temp\ipykernel_23680\721074280.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['sun10'].loc[((test_data['time'] >= 20) | (test_data['time'] <= 4) | ((test_data['time'] == 5) & (test_data['minute'] == 0))) &


CA 지역의 sun10 결측치를 처리합니다.<br>
CA 지역의 긴 구간에 결측치가 있는 것을 확인하여 동일 구간의 CB 지역의 sun10 값으로 대체했습니다.

In [15]:
# 확인한 구간의 시작과 끝 인덱스를 할당
CA_start_idx = train_data.loc[(train_data['stn_id'] == "CA") & 
                              (train_data['year'] == 'K') & 
                              (train_data['month'] == 8) &
                              (train_data['day'] == 26) &
                              (train_data['time'] == 0) &
                              (train_data['minute'] == 10)].index.values[0]
CA_end_idx = train_data.loc[(train_data['stn_id'] == "CA") & 
                            (train_data['year'] == 'K') & 
                            (train_data['month'] == 12) &
                            (train_data['day'] == 23) &
                            (train_data['time'] == 21) &
                            (train_data['minute'] == 0)].index.values[0]

CB_start_idx = train_data.loc[(train_data['stn_id'] == "CB") & 
                              (train_data['year'] == 'K') & 
                              (train_data['month'] == 8) &
                              (train_data['day'] == 26) &
                              (train_data['time'] == 0) &
                              (train_data['minute'] == 10)].index.values[0]
CB_end_idx = train_data.loc[(train_data['stn_id'] == "CB") & 
                            (train_data['year'] == 'K') & 
                            (train_data['month'] == 12) &
                            (train_data['day'] == 23) &
                            (train_data['time'] == 21) &
                            (train_data['minute'] == 0)].index.values[0]
print(CA_start_idx)
print(CA_end_idx)
print(CB_start_idx)
print(CB_end_idx)

2770108
2787369
2822668
2839929


In [16]:
# CA, CB 지역의 시작 지점의 차이로 간격 계산
interval = CB_start_idx - CA_start_idx
print(CB_start_idx - CA_start_idx)
print(CB_end_idx - CA_end_idx)

52560
52560


In [17]:
# CA의 인덱스 및 간격을 사용하여 CB 지역의 sun10 값 할당
# CA가 결측치가 아닌 경우 CA의 sun10값으로 유지
train_na = train_data['sun10'].isna()
for ca_i in range(CA_start_idx, CA_end_idx + 1, 1):
    if train_na.iloc[ca_i] == True:
        train_data['sun10'].iloc[ca_i] = train_data['sun10'].iloc[ca_i + interval]
sum(train_data['sun10'].iloc[CA_start_idx: CA_end_idx + 1].isna())        

C:\Users\JungHwan\AppData\Local\Temp\ipykernel_23680\2971880237.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['sun10'].iloc[ca_i] = train_data['sun10'].iloc[ca_i + interval]


140

R을 사용한 전처리를 위해 데이터를 저장합니다.

In [18]:
train_data.to_csv('../data/train_prepro.csv', index = False)
test_data.to_csv('../data/test_prepro.csv', index = False)